In [2]:
# imports
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go

### Data Processing on WT_pKa

In [ ]:
WT_pka = pd.read_csv('WT_pka.csv')

In [ ]:
WT_pka.info()

In [ ]:
WT_pka.head()

In [ ]:
# get rid of null columns due to file 
WT_pka.drop(WT_pka.columns[-4:], axis = 1, inplace = True)
WT_pka.head()

We are going to drop more columns that we are now not interested in.

In [ ]:
WT_pka.drop(WT_pka.columns[-7:], axis = 1, inplace = True)
WT_pka.head()

In [ ]:
is_NaN = WT_pka.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = WT_pka[row_has_NaN]
print(rows_with_NaN)

# This row does not have an experimental value, so we drop it
WT_pka.dropna(inplace = True)
WT_pka.isna().sum()

In [ ]:
WT_pka['Res ID'] = WT_pka['Res ID'].astype(int)
WT_pka.head()

Process irregular values in Expt. pKa

In [ ]:
# Create a new column 'Greater/Smaller' to keep record of Expt. pKa
WT_pka['Greater/Smaller'] = 0

WT_pka.loc[WT_pka['Expt. pKa'].str.contains(">"), 'Greater/Smaller'] = 1
WT_pka.loc[WT_pka['Expt. pKa'].str.contains("<"), 'Greater/Smaller'] = -1

WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].str.replace('>', '')
WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].str.replace('<', '')
WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].str.replace('~', '')

In [ ]:
# There are two rows with two pKa valus, created a new row to store the second value
print(WT_pka[WT_pka['Expt. pKa'].str.contains(",")])
WT_pka['2nd pKa'] = 0.0
WT_pka[['Expt. pKa','2nd pKa']] = WT_pka['Expt. pKa'].str.split(',',expand=True)
WT_pka.loc[WT_pka['2nd pKa'] == 'None', '2nd pKa'] = '0'
WT_pka['Expt. pKa'] = WT_pka['Expt. pKa'].astype(float)

WT_pka['2nd pKa'] = WT_pka['2nd pKa'].astype(float)
WT_pka['2nd pKa'] = WT_pka['2nd pKa'].fillna(0)

WT_pka.info()

In [ ]:
WT_pka.head()

<hr style="border:1px solid gray"> </hr>

### Data processing on individual proteins (pKa.csv and output.pqr)

#### First create a dataframe for theoretical pka values for future use

In [18]:
# theoretical value of proteins
theo_val = {'ARG': 12.0, 'ASP': 4.0, 'CYS': 9.5, 'GLU': 4.4, 'HIS': 6.3, 
               'LYS': 10.4, 'TYR': 9.6}

df_theo_val = pd.DataFrame(np.array([['ARG', 12.0], ['ASP', 4.0], ['CYS', 9.5], 
                                    ['GLU', 4.4], ['HIS', 6.3], ['LYS', 10.4], ['TYR', 9.6]]), 
                          columns = ['Res Name', 'pKa'])
df_theo_val

,Res Name,pKa
0,ARG,12.0
1,ASP,4.0
2,CYS,9.5
3,GLU,4.4
4,HIS,6.3
5,LYS,10.4
6,TYR,9.6


<hr style="border:1px solid gray"> </hr>

### We use 2ovo as an example

#### Read 2ovo pka file 

In [ ]:
# rearrange pKa.csv, we use 2ovo as an example
df_2ovo = pd.read_csv('sample_data/2ovo/pKa.csv')
df_2ovo.info()

In [ ]:
# We see that all the columns are now in one column, so we need to split them.
df_2ovo[list(df_2ovo.columns)[0].split()] = df_2ovo.iloc[:,0].str.split(expand=True)
df_2ovo.drop(df_2ovo.columns[0], axis = 1, inplace = True)

# Split the Res ID and Res Name from ResName
# "(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)" split digits and chars
df_2ovo[['Res Name', 'Res ID', 'Chain']] = df_2ovo.iloc[:,0].str.split("(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)", expand=True)
df_2ovo.drop(df_2ovo.columns[0], axis = 1, inplace = True)
df_2ovo['Res ID'] = df_2ovo['Res ID'].astype(int)
df_2ovo = df_2ovo[list(df_2ovo.columns)[-3:-1]+ list(df_2ovo.columns)[0:-3]]
df_2ovo = df_2ovo[list(df_2ovo.columns)[0:3]]

df_2ovo.head()

In [ ]:
# Merge with theoretical values
df_2ovo.rename(columns={"pKa": "Expt. pKa"}, inplace=True)
df_2ovo = pd.merge(df_2ovo, df_theo_val, on=['Res Name'], how='inner')
df_2ovo

#### Read 2ovo pqr file 

In [ ]:
file = open('sample_data/2ovo/output.pqr', 'r')
lines = file.readlines()
lines = lines[:-1]
file.close()
column_names = ['Res ID', 'x', 'y', 'z', 'Charge', 'Radius']
df_2ovo_pqr = pd.DataFrame(columns=column_names)
target_IDs = list(df_2ovo['Res ID'].unique().astype(int))
print(target_IDs)
i = 0
for line in lines:
    line = line.strip().split()
    if int(line[5]) in target_IDs:
        df_2ovo_pqr.loc[i] = line[5:] 
        i += 1
df_2ovo_pqr['Res ID'] = df_2ovo_pqr['Res ID'].astype(int)
df_2ovo_pqr[['x', 'y', 'z', 'Charge', 'Radius']] = df_2ovo_pqr[['x', 'y', 'z', 'Charge', 'Radius']].astype(float)
df_2ovo_pqr.head()

In [ ]:
df_2ovo = pd.merge(df_2ovo, df_2ovo_pqr, on=['Res ID'], how='inner')
df_2ovo.head()

In [ ]:

fig = go.Figure()

res_IDs = list(df_2ovo['Res ID'].unique())
data = []

for ID in res_IDs:
    res_name = list(df_2ovo.loc[(df_2ovo['Res ID']) == ID,'Res Name'].unique())[0]
    trace = go.Scatter3d(
        x=df_2ovo.loc[(df_2ovo['Res ID']) == ID,'x'],
        y=df_2ovo.loc[(df_2ovo['Res ID']) == ID,'y'],
        z=df_2ovo.loc[(df_2ovo['Res ID']) == ID,'z'],

        mode='markers',
        marker=dict(
            size=3,
            colorscale='Viridis',   
        ),
        name= res_name+' '+str(ID),

        # list comprehension to add text on hover
        text= [f"x: {a}<br>y: {b}<br>z: {c}" for a,b,c in list(zip(df_2ovo['x'], df_2ovo['y'], df_2ovo['z']))],
        # if you do not want to display x,y,z
        hoverinfo='text'
    )
    fig.add_trace(trace)
    data.append(trace)

layout = dict(title = 'TEST',)

F = dict(data=data, layout=layout)
py.offline.plot(F, filename = 'Test.html')


<hr style="border:1px solid gray"> </hr>

### For any PDBID

In [15]:
def read_csv(PDBID):
    df_PDB_csv = pd.read_csv('sample_data/' + PDBID.lower() + '/pKa.csv')
    
    # We see that all the columns are now in one column, so we need to split them.
    df_PDB_csv[list(df_PDB_csv.columns)[0].split()] = df_PDB_csv.iloc[:,0].str.split(expand=True)
    df_PDB_csv.drop(df_PDB_csv.columns[0], axis = 1, inplace = True)

    # Split the Res ID and Res Name from ResName
    # "(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)" split digits and chars
    df_PDB_csv[['Res Name', 'Res ID', 'Chain']] = df_PDB_csv.iloc[:,0].str.split("(?<=\\D)(?=\\d)|(?<=\\d)(?=\\D)", expand=True)
    df_PDB_csv.drop(df_PDB_csv.columns[0], axis = 1, inplace = True)
    df_PDB_csv['Res ID'] = df_PDB_csv['Res ID'].astype(int)
    df_PDB_csv = df_PDB_csv[list(df_PDB_csv.columns)[-3:-1]+ list(df_PDB_csv.columns)[0:-3]]
    df_PDB_csv = df_PDB_csv[list(df_PDB_csv.columns)[0:3]]
    
    # merge with theoretical values
    df_PDB_csv.rename(columns={"pKa": "Expt. pKa"}, inplace=True)
    df_PDB_csv = pd.merge(df_PDB_csv, df_theo_val, on=['Res Name'], how='inner')
    
    return df_PDB_csv

In [16]:
def read_pqr(PDBID, df_PDB_csv = None, flag = False):
    file = open('sample_data/' + PDBID.lower() + '/output.pqr', 'r')
    lines = file.readlines()
    lines = lines[:-1]
    file.close()
    
    column_names = ['Atom Name', 'Res Name', 'Res ID', 'x', 'y', 'z', 'Charge', 'Radius']
    df_PDB_pqr = pd.DataFrame(columns=column_names)
    if flag:
        target_IDs = list(df_PDB_csv['Res ID'].unique().astype(int))

    i = 0
    
    # find corresponding res ID in pqr file
    for line in lines:
        line = line.strip().split()
        if len(line) == 11:
            if flag == False:
                df_PDB_pqr.loc[i] = [line[2]] + [line[3]] + line[5:]
            elif ((flag) & (int(line[5]) in target_IDs)):
                df_PDB_pqr.loc[i] = [line[2]] + [line[3]] + line[5:]
            i += 1
            
    # convert datatype
    df_PDB_pqr['Res ID'] = df_PDB_pqr['Res ID'].astype(int)
    df_PDB_pqr[['x', 'y', 'z', 'Charge', 'Radius']] = df_PDB_pqr[['x', 'y', 'z', 'Charge', 'Radius']].astype(float)
    df_PDB_pqr.head()
    return df_PDB_pqr
    

## Visualization on a protein

In [130]:
def plot_PDB(PDBID, df_PDB):
    
    fig = go.Figure()

    res_IDs = list(df_PDB['Res ID'].unique())
    data = []

    for ID in res_IDs:
        res_name = list(df_PDB.loc[(df_PDB['Res ID']) == ID,'Res Name'].unique())[0]
        trace = go.Scatter3d(
            x=df_PDB.loc[(df_PDB['Res ID']) == ID,'x'],
            y=df_PDB.loc[(df_PDB['Res ID']) == ID,'y'],
            z=df_PDB.loc[(df_PDB['Res ID']) == ID,'z'],

            mode='markers',
            marker=dict(
                size=3,
                colorscale='Viridis',   
            ),
            name = res_name + ' ' + str(ID),
            # list comprehension to add text on hover
            text = [f"x: {a}<br>y: {b}<br>z: {c}<br>res: {d}" 
                   for a,b,c,d in list(zip(df_PDB['x'], df_PDB['y'], df_PDB['z'], [res_name + ' ' + str(ID)]*len(df_PDB['x']))) ],
            # if you do not want to display x,y,z
            hoverinfo='text'
        )
        fig.add_trace(trace)
        data.append(trace)

    layout = dict(title = PDBID.upper(),)

    F = dict(data=data, layout=layout)
    py.offline.plot(F, filename = 'sample_graphs/' +PDBID + '.html')
    

In [144]:
def analyze_PDB(PDBID):
    df_PDB_csv = read_csv(PDBID)
    df_PDB_pqr = read_pqr(PDBID, df_PDB_csv, flag = True)
    # merge csv and pqr
    df_PDB = pd.merge(df_PDB_csv, df_PDB_pqr, on=['Res ID', 'Res Name'], how='inner')
    plot_PDB(PDBID, df_PDB)

In [157]:
sample_data = ['1bf4', '1bpi', '1igd', '1pga', '1pgb', '2ci2', '2ovo', '2qmt', '3ebx', '4pti']
for PDBID in sample_data:
    analyze_PDB(PDBID)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507 entries, 0 to 506
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Res Name   507 non-null    object 
 1   Res ID     507 non-null    int64  
 2   Expt. pKa  507 non-null    object 
 3   pKa        507 non-null    object 
 4   Atom Name  507 non-null    object 
 5   x          507 non-null    float64
 6   y          507 non-null    float64
 7   z          507 non-null    float64
 8   Charge     507 non-null    float64
 9   Radius     507 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 43.6+ KB


<hr style="border:1px solid gray"> </hr>

### Preprocess Data for Prediction
- Purpose here is to analyze the same amino acid and see if there's a pattern even amoung different proteins
- We will first use LYS and our sample_data as an experiment. Things we need to do:
    - Calculate Coulomb force on each LYS atom from all the other atoms (since looping in python is terrible, we might use matrix?)
    - We need to extract all rows of LYS from our sample proteins
    - The features that we are interested in are 'Atom name', 'Res Name', 'Res ID', 'x', 'y', 'z', 'Charge', 'Radius'
    - One observation is that for the same atom, its charge and radius are the same. 
        - Need to confirm if it's true
        - We could analyze whether the prediction behaves differently if we replace the numerical value with only the atom if we decide whether it is discrete or continuous

In [37]:
target_AA = 'ARG'
sample_data = ['1bf4', '1bpi', '1igd', '1pga', '1pgb', '2ci2', '2ovo', '2qmt', '3ebx', '4pti']

In [27]:
def extract_with_AA(PDBID, Amino_Acid):
    df_PDB_pqr_all = read_pqr(PDBID)
    df_PDB_csv = read_csv(PDBID)
    df_PDB_pqr = read_pqr(PDBID, df_PDB_csv)
    df_PDB = pd.merge(df_PDB_csv, df_PDB_pqr, on=['Res ID', 'Res Name'], how='inner')
    target_rows = df_PDB.loc[(df_PDB['Res Name'] == Amino_Acid)]
    return target_rows, df_PDB_pqr_all

In [28]:
def calculate_coulomb_force(target_row, df_PDB_pqr_all):
    coulomb_force = 0
    for index, row in df_PDB_pqr_all.iterrows():
        dist = np.sqrt((row['x'] - target_row['x'])**2 + (row['y'] - target_row['y'])**2 + (row['z'] - target_row['z'])**2)
        if dist == 0:
            continue
        coulomb_force = coulomb_force + row['Charge']/dist
    return coulomb_force

In [29]:
def arrange_df(PDBID):
    df_PDB, df_PDB_pqr_all = extract_with_AA(PDBID, target_AA)
    df_PDB['Columb Force'] = 0
    for index, row in df_PDB.iterrows():
        df_PDB.loc[index, 'Columb Force'] = calculate_coulomb_force(row, df_PDB_pqr_all)
    df_PDB['PDBID'] = PDBID.upper()
    return df_PDB

In [30]:
# find the same Amino Acid in all PDB
def concat_DFs(sample_data):
    first = True
    for PDBID in sample_data:
        df_PDB = arrange_df(PDBID)
        # rearrange columns
        df_PDB = df_PDB[[list(df_PDB.columns)[-1]] + [list(df_PDB.columns)[4]] + 
                          list(df_PDB.columns)[0:2] + list(df_PDB.columns)[-7:-1] + 
                          list(df_PDB.columns)[2:4]]
        if first:
            df_AA = pd.concat([df_PDB])
            first = False
        else:
            df_AA = pd.concat([df_AA, df_PDB])
    return df_AA

In [38]:
df_LYS = concat_DFs(sample_data)

In [39]:
pd.set_option('display.max_rows', None)
df_LYS.reset_index(drop=True)
df_LYS = df_LYS.reset_index(drop=True)

In [40]:
df_LYS

,PDBID,Atom Name,Res Name,Res ID,x,y,z,Charge,Radius,Columb Force,Expt. pKa,pKa
0,1BF4,N,ARG,25,-4.289,21.980,13.379,-0.3479,1.824,0.421643,12.94,12.0
1,1BF4,H,ARG,25,-5.714,21.560,13.171,0.2747,0.600,-0.057897,12.94,12.0
2,1BF4,CA,ARG,25,-4.018,23.372,13.753,-0.2637,1.908,0.335637,12.94,12.0
3,1BF4,HA,ARG,25,-2.974,23.543,13.582,0.1560,1.387,-0.148362,12.94,12.0
4,1BF4,C,ARG,25,-4.304,23.655,15.235,0.7341,1.908,-0.727392,12.94,12.0
5,1BF4,O,ARG,25,-5.309,23.210,15.789,-0.5894,1.661,0.491232,12.94,12.0
6,1BF4,CB,ARG,25,-4.883,24.317,12.893,-0.0007,1.908,0.171951,12.94,12.0
7,1BF4,HB1,ARG,25,-5.897,23.997,13.030,0.0327,1.487,0.182034,12.94,12.0
8,1BF4,HB2,ARG,25,-4.446,24.310,11.914,0.0327,1.487,0.148767,12.94,12.0
9,1BF4,CG,ARG,25,-4.831,25.766,13.367,0.0390,1.908,0.173120,12.94,12.0


In [36]:
len(df_LYS['Res ID'].unique())

36

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>

<hr style="border:1px solid gray"> </hr>